In [51]:
#1#Header
import csv
import numpy as np
import os 
from os import urandom
from keras.models import model_from_json

In [94]:
#2#Defining Global Variables
num_rounds = 20
m = 0
o = 0
counter = 0
k_int = 0
k_int1 = 0

In [95]:
#3#Defining WORDSIZE
def WORD_SIZE():
    return(16);

In [96]:
#4#Defining S-Box
s_box_mapping_np = np.array([12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2], dtype=np.uint8)

def s_box(input_bits):
    input_bits_int = int(input_bits)
    output_bits_int = s_box_mapping_np[input_bits_int]
    return output_bits_int

In [97]:
#5#Defining P-Box
def decimal_to_binary_list(value, num_bits=4):
    return np.array([int(x) for x in format(value, f'0{num_bits}b')], dtype=np.uint8)

def p_box(c_decimal, d_decimal, x_decimal, y_decimal):
    c = decimal_to_binary_list(c_decimal)
    d = decimal_to_binary_list(d_decimal)
    x = decimal_to_binary_list(x_decimal)
    y = decimal_to_binary_list(y_decimal)
    
    e = np.zeros(16, dtype=np.uint8)

    e[0] = d[3]
    e[1] = y[1]
    e[2] = c[1]
    e[3] = x[0]
    e[4] = x[3]
    e[5] = y[2]
    e[6] = c[2]
    e[7] = d[1]
    e[8] = d[0]
    e[9] = x[2]
    e[10] = y[3]
    e[11] = c[0]
    e[12] = c[3]
    e[13] = d[2]
    e[14] = x[1]
    e[15] = y[0]

    return e

In [98]:
#6#Defining L-Box
def l_box(f):

    h = np.zeros(16, dtype=np.uint8)
    h[0] = f[0]
    h[1] = f[8]
    h[2] = f[7]
    h[3] = f[15]
    h[4] = f[1]
    h[5] = f[9]
    h[6] = f[6]
    h[7] = f[14]
    h[8] = f[2]
    h[9] = f[10]
    h[10] = f[5]
    h[11] = f[13]
    h[12] = f[3]
    h[13] = f[11]
    h[14] = f[4]
    h[15] = f[12]
    #print("H:", h)
    return h

In [99]:
#7#Defining F-function for Right Side of Plaintext
def binary_array_to_integer(output):
    int_output = ''.join(map(str, output))
    return int(int_output, 2)

def to_binary(value, bits):
    return format(value, f'0{bits}b')

def f_function(x, key, d):
    q=0
    global m, counter, k_int
    #print("X:", x)
    if isinstance(x, int):
        x = [x]
    input_parts = np.zeros((len(x), 4), dtype=np.uint16)
    for i, val in enumerate(x):
        input_parts[i] = np.array([val >> 12, (val >> 8) & 0xF, (val >> 4) & 0xF, val & 0xF])
    #print("F_FUNCTION")
    #print(input_parts)
    s_box_outputs = np.array([[s_box(element) for element in part] for part in input_parts])
    #print("S-box:", s_box_outputs)
    p_box_outputs = np.zeros((len(x), 1, 16), dtype=np.uint8)
    for i in range(len(x)):
        p_box_outputs[i] = np.array(p_box(s_box_outputs[i][0], s_box_outputs[i][1], s_box_outputs[i][2], s_box_outputs[i][3]))
    #print("P-box:", p_box_outputs)
    final_outputs = np.zeros(len(x), dtype=np.uint32)
    #print(len(x))
    for i in range(len(x)):
        #print(len(x))
        final_output = np.array(l_box(p_box_outputs[i][0]))
        k = key[q][(m+1) % 4]
        #print("final_output:", final_output)
        #print("Key:", k)
        if (counter > 1):
            #print("counter:", counter)
            k_bin, k_int = subsequent_key(k_int)
            #print("Key in binary:", k_bin)
            #print("k in int", k_int)
            output = final_output ^ k_bin
        else:
            k = to_binary(k,16)
            k = np.array([int(bit) for bit in k])
            #print("k", k)
            output = final_output ^ k
        #print("XORING output:", output)
        output = binary_array_to_integer(output)
        final_outputs[i] = output
        q +=1 
    #print("Final output:", final_outputs)
    if (m < 2):
            m +=2
    else:
            m = 0
            
    #print("_______________________________________________________________")
    return final_outputs

In [100]:
def to_binary(value, bits):
    return format(value, f'0{bits}b')

def binary_array_to_integer(output):
    int_output = ''.join(map(str, output))
    return int(int_output, 2)

def subsequent_key(x):
    #x = [x]
    if isinstance(x, int):
        x = [x]
    #print("sub key", x)
    input_parts = np.zeros((len(x), 4), dtype=np.uint16)
    for i, val in enumerate(x):
        input_parts[i] = np.array([val >> 12, (val >> 8) & 0xF, (val >> 4) & 0xF, val & 0xF])
    #print("input_part", input_parts)
    s_box_outputs = np.array([[s_box(element) for element in part] for part in input_parts])
    #print("S-box:", s_box_outputs)
    p_box_outputs = np.zeros((len(x), 1, 16), dtype=np.uint8)
    for i in range(len(x)):
        p_box_outputs[i] = np.array(p_box(s_box_outputs[i][0], s_box_outputs[i][1], s_box_outputs[i][2], s_box_outputs[i][3]))
    #print("P-box:", p_box_outputs)
    bin_output = np.zeros(len(x), dtype=np.uint16)
    final_output = np.zeros(len(x), dtype=np.uint16)
    for i in range(len(x)):
        bin_output = np.array(l_box(p_box_outputs[i][0]))
        #print(bin_output)
        #final_outputs[i] = final_output
        output = binary_array_to_integer(bin_output)
        #print(output)
        final_output[i] = output
        
    #print("final_outputs:", final_outputs)
    return bin_output, final_output

In [101]:
#8#Defining F-function for Left Side of Plaintext
def binary_array_to_integer(output):
    int_output = ''.join(map(str, output))
    return int(int_output, 2)

def ff_function(x, key, d):
    q=0
    global o, counter, k_int1
    if isinstance(x, int):
        x = [x]
        
    input_parts = np.zeros((len(x), 4), dtype=np.uint16)
    for i, val in enumerate(x):
        input_parts[i] = np.array([val >> 12, (val >> 8) & 0xF, (val >> 4) & 0xF, val & 0xF])
    #print("FF_FUNCTION")
    #print(input_parts)
    s_box_outputs = np.array([[s_box(element) for element in part] for part in input_parts])
    #print("S-box:", s_box_outputs)
    p_box_outputs = np.zeros((len(x), 1, 16), dtype=np.uint8)
    for i in range(len(x)):
        p_box_outputs[i] = np.array(p_box(s_box_outputs[i][0], s_box_outputs[i][1], s_box_outputs[i][2], s_box_outputs[i][3]))
    #print("P-box:", p_box_outputs)
    final_outputs = np.zeros(len(x), dtype=np.uint32)
    #print(len(x))
    for i in range(len(x)):
        #print(len(x))
        final_output = np.array(l_box(p_box_outputs[i][0]))
        k = key[q][o % 4]
        #print("final_output:", final_output)
        #print("Key in int:", k)
        if (counter > 1):
            k_bin, k_int1 = subsequent_key(k_int1)
            #print("Key in binary:", k_bin)
            #print("k", k_int)
            output = final_output ^ k_bin
        else:
            k = to_binary(k,16)
            k = np.array([int(bit) for bit in k])
            #print("k", k)
            output = final_output ^ k
        #print("XORING output:", output)
        output = binary_array_to_integer(output)
        final_outputs[i] = output
        q +=1 
    counter += 1
    #print("Final output:", final_outputs)
    if (o < 2):
            o +=2
    else:
            o = 0
    #print("_______________________________________________________________")
    return final_outputs

In [116]:
#9#Convert the ciphertext pairs into Binary array
def convert_to_binary(row):
    bin_array = np.zeros(32, dtype=np.uint8)
    binary_str = format(row[0], '016b') + format(row[1], '016b')
    for i, b in enumerate(binary_str):
        bin_array[i] = int(b)
    return bin_array

In [103]:
#10#Encryption Function
def lcb_encrypt(plaintext, key, rounds, d):
    
    left_plaintext = np.uint16(plaintext[0])
    right_plaintext = np.uint16(plaintext[1])
    L, R = left_plaintext, right_plaintext

    n = 0
    
    while n < rounds:
        L, R = f_function(R, key, d), ff_function(L, key, d)
        n += 1
    print("Encryption done per round") 
    #print(rounds)
    #print(n)
    return (L, R)

In [104]:
#11#Fuction for generation of keys
import random

def generate_hex_keys(num_keys, length=16):
    hex_chars = "0123456789ABCDEF"
    keys_str = ["".join(random.choices(hex_chars, k=length)) for _ in range(num_keys)]

    return keys_str

def generate_round_keys(num_keys):
    random_keys_hex = generate_hex_keys(num_keys)
    #random_keys_hex = ['D63A529ECC92D353', '563A529ECC92D353', '163A529ECC92D353', 'D67AD296CC92DB53', '76BA569EDC9BD353']
    #random_keys_hex = ['163A529D687529EC']
    round_keys = []
    
    for random_key_hex in random_keys_hex:
        random_key = int(random_key_hex, 16)

        K1 = (random_key >> 48) & 0xFFFF
        K2 = (random_key >> 32) & 0xFFFF
        K3 = (random_key >> 16) & 0xFFFF
        K4 = random_key & 0xFFFF
        
        #k1_bin = to_binary(K1, 16)
        #k2_bin = to_binary(K2, 16)
        #k3_bin = to_binary(K3, 16)
        #k4_bin = to_binary(K4, 16)

        #k1_np_array = np.array([int(bit) for bit in k1_bin])
        #k2_np_array = np.array([int(bit) for bit in k2_bin])
        #k3_np_array = np.array([int(bit) for bit in k3_bin])
        #k4_np_array = np.array([int(bit) for bit in k4_bin])

        round_key = np.array([K1, K2, K3, K4])
        round_keys.append(round_key)
    round_key = np.array(round_keys)
    #print("Key generation done:", round_keys)
    return round_key

In [133]:
#12#Make dataset

def make_train_data(n, nr, var):
  global counter

  nonce = np.frombuffer(urandom(2), dtype=np.uint16);
  cownter = np.arange(0,1023, dtype=np.uint16)
  input = (nonce.astype(np.uint32) << 16) | cownter.astype(np.uint32)
  plaintext = np.frombuffer(urandom(4*n), dtype=np.uint32);
  #plaintext = [0xEED4B555]
  #plaintext = [0xCED4B5C6, 0xCED4B5C6, 0xCED4B5C6, 0xCED4B5C6, 0xCED4B5C6]
  plain0l = np.empty(n, dtype=np.uint16)
  plain0r = np.empty(n, dtype=np.uint16)
  plaintext0l = np.frombuffer(urandom(2*n), dtype=np.uint16)
  plaintext0r = np.frombuffer(urandom(2*n), dtype=np.uint16)
  
  for i in range(n):
    plain0l[i] = (input[i] >> 16) & 0xffff
    plain0r[i] = input[i] & 0xffff
  
  round_keys = generate_round_keys(1)
  
  round_key = np.repeat(round_keys, 1024, axis=0)
    
  ctdata0l, ctdata0r = lcb_encrypt((plain0l, plain0r), round_key, nr, n)
  ciphertext0l = ctdata0l ^ plaintext0l
  print(ciphertext0l[0])
  ciphertext0r = ctdata0r ^ plaintext0r
  print(ciphertext0r[0])
  ctdata = np.vstack((ciphertext0l, ciphertext0r)).T
  
  X = np.array([convert_to_binary(row) for row in ctdata])
  print(X[0])
  X = X.reshape(1, -1)
  
  filename = f"output_{var}.txt"

# Open the file in write mode (overwrites existing file or creates a new one)
  with open(filename, 'w') as file:
    # Convert the array to a string without spaces
    X_string = ''.join(X.astype(str)[0])

    # Write the string to the file
    file.write(X_string)
  
  """
  with open("Dataset_NewP.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["plain0l", "plain0r", "plain1l", "plain1r","Y"])
    for i in range(n):
        writer.writerow([plain0l[i], plain0r[i], plain1l[i], plain1r[i],Y[i]])

    with open("Dataset_NewC.csv", "w", newline='') as f:
      writer = csv.writer(f)
      writer.writerow(["ctdata0l", "ctdata0r", "ctdata1l", "ctdata1r","Y"])
      for i in range(n):
          writer.writerow([ctdata0l[i], ctdata0r[i], ctdata1l[i], ctdata1r[i],Y[i]])
  """
  return(X);

In [134]:
for i in range (1000000):
    make_train_data(1023,20,i)

Encryption done per round
25812
59196
[0 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 0]
Encryption done per round
65099
11722
[1 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0]
Encryption done per round
35646
42735
[1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1]
Encryption done per round
1385
17212
[0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0]
Encryption done per round
53116
9840
[1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 1 1 1 0 0 0 0]
Encryption done per round
46666
41535
[1 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 1 1]
Encryption done per round
56968
29415
[1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1]
Encryption done per round
19138
32283
[0 1 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1]
Encryption done per round
29708
61356
[0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0]
Encryption done per round
60847
3956
[1 1 1 0 1 1 0 1 1 0 1 0 1 1 

In [136]:
# Define the number of files and bits per row
num_files = 1000000
bits_per_row = 32736

# Initialize the merged array
merged_array = []

# Iterate over the files and read the content
for i in range(num_files):
    file_name = f"output_{i}.txt"  # Adjust the file name pattern if needed
    i +=1
    with open(file_name, 'r') as file:
        content = file.read()
        merged_array.append(content)

# Convert the merged array into a single string with newline characters
merged_string = '\n'.join(merged_array)

# Write the merged string to the output file
output_file = "merged_output.txt"
with open(output_file, 'w') as file:
    file.write(merged_string)
